[Reference](https://medium.com/@HeCanThink/functime-a-python-library-for-efficient-time-series-feature-extraction-and-forecasting-5339f9a26b15)

In [1]:
pip install functime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.8/26.8 MB 40.9 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.20.2
    Uninstalling polars-0.20.2:
      Successfully uninstalled polars-0.20.2


In [2]:
# pip install "functime[llm,lgb]"

In [3]:
#Importing the libraries and the dataset
import polars as pl

from functime.plotting import plot_forecasts, plot_panel
from functime.preprocessing import (
    boxcox,
    deseasonalize_fourier,
    detrend,
    diff,
    fractional_diff,
    scale,
    yeojohnson,
)
data = pl.read_parquet("https://github.com/TracecatHQ/functime/raw/main/data/commodities.parquet")
entity_col, time_col, target_col = data.columns
data.head()

commodity_type,time,price
str,datetime[ns],f64
"""Aluminum""",1960-01-01 00:00:00,511.47
"""Aluminum""",1960-02-01 00:00:00,511.47
"""Aluminum""",1960-03-01 00:00:00,511.47
"""Aluminum""",1960-04-01 00:00:00,511.47
"""Aluminum""",1960-05-01 00:00:00,511.47


In [4]:
data.get_column("commodity_type").n_unique()

71

In [5]:
most_volatile_commodities = (
    data.group_by(entity_col)
    .agg((pl.col(target_col).std() / pl.col(target_col).mean()).alias("cv"))
    .top_k(k=4, by="cv")
)
most_volatile_commodities

commodity_type,cv
str,f64
"""Natural gas, E…",1.341444
"""Phosphate rock…",1.104353
"""Potassium chlo…",1.070662
"""Crude oil, Dub…",0.987116


In [12]:
# selected = most_volatile_commodities.get_column(entity_col)
# y = data.filter(pl.col(entity_col).is_in(selected))
# figure = plot_panel(y=y, height=800, width=1000,replace=True)
# figure.show()

In [13]:
# transformer = detrend(freq="1mo", method="linear")
# y_detrended = y.pipe(transformer).collect()
# figure = plot_forecasts(
#     y_true=y, y_pred=y_detrended.group_by(entity_col).tail(64), height=800, width=1000
# )
# figure.show()

In [15]:
# m4_data = pl.read_parquet("https://github.com/TracecatHQ/functime/raw/main/data/m4_1w_train.parquet")
# m4_entity_col, m4_time_col, m4_target_col = m4_data.columns
# y_m4 = m4_data.filter(pl.col(m4_entity_col).is_in(["W174", "W175", "W176", "W178"]))
# figure = plot_panel(y=y_m4, height=800, width=1000)
# figure.show()

In [17]:
# # Fourier Terms
# transformer = deseasonalize_fourier(sp=12, K=3)
# y_deseasonalized = y_m4.pipe(transformer).collect()
# y_seasonal = transformer.state.artifacts["X_seasonal"].collect()
# figure = plot_panel(
#     y=y_seasonal.group_by(m4_entity_col).tail(64), height=800, width=1000
# )
# figure.show()

In [18]:
# y_deseasonalized = y_m4.pipe(transformer).collect()
# y_original = transformer.invert(y_deseasonalized).collect()
# figure = plot_panel(
#     y=y_original.group_by(m4_entity_col).tail(64), height=800, width=1000
# )
# figure.show()

In [19]:
# transformer = diff(order=1)
# y_diff = y.pipe(transformer).collect()
# figure = plot_forecasts(
#     y_true=y, y_pred=y_diff.group_by(entity_col).tail(64), height=800, width=1000
# )
# figure.show()

In [20]:
# transformer = fractional_diff(d=0.3, min_weight=1e-3)
# y_diff = y.pipe(transformer).collect()
# figure = plot_forecasts(
#     y_true=y, y_pred=y_diff.group_by(entity_col).tail(64), height=800, width=1000
# )
# figure.show()

In [21]:
# transformer = diff(order=1, sp=12)
# y_seas_diff = y.pipe(transformer).collect()
# figure = plot_forecasts(
#     y_true=y, y_pred=y_seas_diff.group_by(entity_col).tail(64), height=800, width=1000
# )
# figure.show()

In [22]:
# transformer = scale(use_mean=True, use_std=True)
# y_scaled = y_m4.pipe(transformer).collect()
# figure = plot_panel(y=y_scaled.group_by(m4_entity_col).tail(64), height=800, width=1000)
# figure.show()

In [23]:
# transformer = boxcox(method="mle")
# y_boxcox = y.pipe(transformer).collect()
# figure = plot_panel(y=y_boxcox.group_by(entity_col).tail(64), height=800, width=1000)
# figure.show()

In [24]:
# transformer = yeojohnson()
# y_yeojohnson = y.pipe(transformer).collect()
# figure = plot_panel(
#     y=y_yeojohnson.group_by(entity_col).tail(64), height=800, width=1000
# )
# figure.show()

In [25]:
import polars as pl
import numpy as np
from functime.feature_extractors import FeatureExtractor, binned_entropy

# Load commodities price data
y = pl.read_parquet("https://github.com/TracecatHQ/functime/raw/main/data/commodities.parquet")

# Get column names ("commodity_type", "time", "price")
entity_col, time_col, value_col = y.columns

# Extract a single feature from a single time-series
binned_entropy = binned_entropy(
    pl.Series(np.random.normal(0, 1, size=10)),
    bin_count=10
)

# Also works on LazyFrames with query optimization
features = (
    pl.LazyFrame({
        "index": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        "value": np.random.normal(0, 1, size=10)
    })
    .select(
        binned_entropy=pl.col("value").ts.binned_entropy(bin_count=10),
        lempel_ziv_complexity=pl.col("value").ts.lempel_ziv_complexity(threshold=3),
        longest_streak_above_mean=pl.col("value").ts.longest_streak_above_mean(),
    )
    .collect()
)

# Extract features blazingly fast on many stacked time-series using `group_by`
features = (
    y.group_by(entity_col)
    .agg(
        binned_entropy=pl.col(value_col).ts.binned_entropy(bin_count=10),
        lempel_ziv_complexity=pl.col(value_col).ts.lempel_ziv_complexity(threshold=3),
        longest_streak_above_mean=pl.col(value_col).ts.longest_streak_above_mean(),
    )
)

# Extract features blazingly fast on windows of many time-series using `group_by_dynamic`
features = (
    # Compute rolling features at yearly intervals
    y.group_by_dynamic(
        time_col,
        every="12mo",
        by=entity_col,
    )
    .agg(
        binned_entropy=pl.col(value_col).ts.binned_entropy(bin_count=10),
        lempel_ziv_complexity=pl.col(value_col).ts.lempel_ziv_complexity(threshold=3),
        longest_streak_above_mean=pl.col(value_col).ts.longest_streak_above_mean(),
    )
)

In [26]:
import polars as pl
from functime.cross_validation import train_test_split
from functime.seasonality import add_fourier_terms
from functime.forecasting import linear_model
from functime.preprocessing import scale
from functime.metrics import mase

# Load commodities price data
y = pl.read_parquet("https://github.com/TracecatHQ/functime/raw/main/data/commodities.parquet")
entity_col, time_col = y.columns[:2]

# Time series split
y_train, y_test = y.pipe(train_test_split(test_size=3))

# Fit-predict
forecaster = linear_model(freq="1mo", lags=24)
forecaster.fit(y=y_train)
y_pred = forecaster.predict(fh=3)

# functime functional design
# fit-predict in a single line
y_pred = linear_model(freq="1mo", lags=24)(y=y_train, fh=3)

# Score forecasts in parallel
scores = mase(y_true=y_test, y_pred=y_pred, y_train=y_train)

# Forecast with target transforms and feature transforms
forecaster = linear_model(
    freq="1mo",
    lags=24,
    target_transform=scale(),
    feature_transform=add_fourier_terms(sp=12, K=6)
)

# Forecast with exogenous regressors!
# Just pass them into X
X = (
    y.select([entity_col, time_col])
    .pipe(add_fourier_terms(sp=12, K=6)).collect()
)
X_train, X_future = y.pipe(train_test_split(test_size=3))
forecaster = linear_model(freq="1mo", lags=24)
forecaster.fit(y=y_train, X=X_train)
y_pred = forecaster.predict(fh=3, X=X_future)
y_pred.head()

commodity_type,time,price
str,datetime[μs],f64
"""Copper""",2023-01-01 00:00:00,9037.948242
"""Copper""",2023-02-01 00:00:00,8936.589844
"""Copper""",2023-03-01 00:00:00,8856.30957
"""Banana, US""",2023-01-01 00:00:00,1.680122
"""Banana, US""",2023-02-01 00:00:00,1.660122
